# Q-Learning in a Flexible NxN GridWorld with Obstacles

This notebook demonstrates how to compute an optimal policy using Q-values (value iteration) in a deterministic GridWorld environment. It supports:
- Arbitrary grid sizes (`NxN`)
- Configurable obstacles
- Any custom goal position
- Visualization of the learned policy and path from any start point


In [38]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Grid settings
grid_world = [6, 6]  # rows, cols
GOAL_REWARD = 10.0
STEP_PENALTY = -1.0
OBSTACLES = [(1, 1), (2, 1), (3, 1), (4, 3), (4, 4), (1, 4), (2, 4), (3, 4)]
GOAL_STATE = (0, 5)


## Environment Construction
Create a list of valid states (excluding obstacles), and set up the transition and reward matrices.


In [39]:
actions = np.array(['up', 'down', 'left', 'right'])
action_moves = {'up': (-1, 0), 'down': (1, 0), 'left': (0, -1), 'right': (0, 1)}

def build_environment(grid_size, obstacles, goal_state):
    states = [(j, i) for j in range(grid_size[0]) for i in range(grid_size[1]) if (j, i) not in obstacles]
    assert goal_state in states, "Goal state must be valid and not an obstacle."
    goal_index = states.index(goal_state)
    return states, goal_index

states, GOAL_INDEX = build_environment(grid_world, OBSTACLES, GOAL_STATE)

## Q-Value Initialization and Update (Value Iteration)
This uses value iteration to compute the Q-values based on a deterministic transition model.


In [40]:
Q_values = np.zeros((len(states), len(actions)))
transition_probability = np.zeros((len(states), len(actions), len(states)))
reward_matrix = np.zeros((len(states), len(actions), len(states)))

for i, s in enumerate(states):
    if i == GOAL_INDEX:
        continue
    for j, a in enumerate(actions):
        move = action_moves[a]
        next_state = (s[0] + move[0], s[1] + move[1])
        next_state_val = (max(0, min(next_state[0], grid_world[0] - 1)),
                          max(0, min(next_state[1], grid_world[1] - 1)))
        if next_state_val in OBSTACLES:
            next_state_val = s
        sp_index = states.index(next_state_val)
        transition_probability[i, j, sp_index] = 1.0
        reward_matrix[i, j, sp_index] = GOAL_REWARD if sp_index == GOAL_INDEX else STEP_PENALTY

gamma = 0.90
for _ in range(50):
    q_old = Q_values.copy()
    for s in range(len(states)):
        for a in range(len(actions)):
            Q_values[s, a] = sum([
                transition_probability[s, a, sp] *
                (reward_matrix[s, a, sp] + gamma * np.max(q_old[sp]))
                for sp in range(len(states))
            ])


## Visualizing the Optimal Policy
Arrows indicate the optimal action to take from each cell, except obstacles and the goal.


In [ ]:
def plot_arrows(Q_values, grid_world, actions, states, goal_state, path=None, OBSTACLES=None):
    action_symbol_map = {'up': '↑','down': '↓','left': '←','right': '→'}
    action_symbols = [action_symbol_map[action] for action in actions]

    policy_indices = np.argmax(Q_values, axis=1)
    policy_symbols = [action_symbols[i] for i in policy_indices]

    # FIX: Replace the goal state symbol instead of inserting
    goal_idx = states.index(goal_state)
    policy_symbols[goal_idx] = '\U0001F3C6'  # 🏆 Trophy symbol

    symbol_grid = []
    OBSTACLE_SYMBOL = '\u2588'
    START_STATE_SYMBOL = "\u26F3"
    idx = 0

    for i in range(grid_world[0]):
        row = []
        for j in range(grid_world[1]):
            if OBSTACLES and (i, j) in OBSTACLES:
                row.append(OBSTACLE_SYMBOL)
            elif (i, j) in states:
                row.append(policy_symbols[idx])
                idx += 1
            else:
                row.append(' ')  # Empty if not in states (e.g., removed due to obstacle)
        symbol_grid.append(row)

    # Overlay path if provided
    if path:
        for step_num, (i, j) in enumerate(path):
            if (i, j) == goal_state:
                continue
            elif (i, j) == path[0]:
                symbol_grid[i][j] = START_STATE_SYMBOL  # Start flag
            else:
                symbol_grid[i][j] = str(step_num % 10)  # Step number

    print("Optimal Policy Grid:")
    for row in symbol_grid:
        print('    '.join(row))
    print()



def plot_grid_world(states, grid_size, obstacles, goal, path=None, Q_values=None, actions=None):
    GOAL_SYMBOL = '\U0001F3C6'
    fig, ax = plt.subplots(figsize=(grid_size[1], grid_size[0]))
    ax.set_xlim(0, grid_size[1])
    ax.set_ylim(0, grid_size[0])
    ax.set_xticks(range(grid_size[1] + 1))
    ax.set_yticks(range(grid_size[0] + 1))
    ax.set_aspect('equal')
    ax.invert_yaxis()

    # Draw grid cells
    for i in range(grid_size[0]):
        for j in range(grid_size[1]):
            rect = patches.Rectangle((j, i), 1, 1, linewidth=1, edgecolor='gray', facecolor='white')
            ax.add_patch(rect)
            if (i, j) in obstacles:
                rect.set_facecolor('black')
            elif (i, j) == goal:
                ax.text(j+0.5, i+0.5, "G:\u2605", ha='center', va='center', fontsize=20, color='red')

    # Draw policy arrows
    if Q_values is not None and actions is not None:
        action_to_vector = {
            'up': (0, -0.3),
            'down': (0, 0.3),
            'left': (-0.3, 0),
            'right': (0.3, 0)
        }
        for idx, state in enumerate(states):
            if state == goal:
                continue
            best_action = actions[np.argmax(Q_values[idx])]
            dx, dy = action_to_vector[best_action]
            x, y = state[1] + 0.5, state[0] + 0.5
            ax.arrow(x, y, dx, dy, head_width=0.1, head_length=0.1, fc='blue', ec='blue')

    # Draw path
    if path:
        for idx, (i, j) in enumerate(path):
            if (i, j) != goal:
                ax.text(j + 0.5, i + 0.5, str(idx % 10), ha='center', va='center', fontsize=14, color='green')

    ax.grid(True)
    plt.show()

plot_arrows(Q_values, grid_world, actions, states, goal_state=GOAL_STATE, path=None, OBSTACLES=OBSTACLES)
plot_grid_world(states, grid_world, OBSTACLES, GOAL_STATE, path=None, Q_values=Q_values, actions=actions)

## Simulate a Policy Rollout
Given a start location, roll out the deterministic policy until the goal is reached.


In [ ]:
def simulate_policy(Q_values, states, actions, start_index, goal_index):
    path = [states[start_index]]
    current_index = start_index
    while current_index != goal_index:
        best_action = np.argmax(Q_values[current_index])
        move = action_moves[actions[best_action]]
        next_state = (states[current_index][0] + move[0], states[current_index][1] + move[1])
        next_state = (max(0, min(next_state[0], grid_world[0] - 1)),
                      max(0, min(next_state[1], grid_world[1] - 1)))
        if next_state in OBSTACLES:
            next_state = states[current_index]
        current_index = states.index(next_state)
        path.append(states[current_index])
    return path

START_STATE = (2, 2)
START_INDEX = states.index(START_STATE)
path = simulate_policy(Q_values, states, actions, START_INDEX, GOAL_INDEX)
plot_arrows(Q_values, grid_world, actions, states, goal_state=GOAL_STATE, path=path, OBSTACLES=OBSTACLES)
plot_grid_world(states, grid_world, OBSTACLES, GOAL_STATE, path=path, Q_values=Q_values, actions=actions)